# PyTorch Lightning +CPU/TPU MNIST BackDoor Detection

# Setup

## CPU/TPU
This notebook requires some packages besides pytorch-lightning.

###  Install Colab TPU compatible PyTorch

In [ ]:
# import os
# assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
# !pip install --quiet -U cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

## Import package

In [ ]:
!pip install pytorch_lightning -qq

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchmetrics.functional import accuracy
from torchvision import transforms

# Note - you must have torchvision installed for this example
from torchvision.datasets import MNIST

import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision.datasets import MNIST
from torchvision import transforms
import torch
import numpy as np


## Select Seed

In [ ]:
torch.manual_seed(1)

## Mount to local folder

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd '/content/drive/Othercomputers/MacBookPro/Google-MyDrive/##Research-in-ML/Pitt-ML/BackDoorDetection'

In [ ]:
# %ls

## Define Parameters

In [ ]:
# Define Parameters

# BATCH_SIZE = 1024

FLAGS = {}
FLAGS['datadir'] = "./data"
FLAGS['num_classes'] = 2

FLAGS['backdoor_TargetLabel'] = 0
FLAGS['backdoor_OriginalLabel'] = 1

FLAGS['backdoor_sample_ratio'] = 1.0
FLAGS['TrainValidate_ratio'] = 0.55


# FLAGS['num_classes'] = 3
FLAGS['batch_size'] = 128
FLAGS['num_workers'] = 40
# FLAGS['num_workers'] = 8
FLAGS['learning_rate'] = 0.01
FLAGS['momentum'] = 0.5
FLAGS['num_epochs'] = 10
FLAGS['num_cores'] = 8
FLAGS['log_steps'] = 20
FLAGS['metrics_debug'] = False
FLAGS['saved_model_pth'] = "/saved_model/"

In [ ]:
def visualize_tensor(tensor):
    to_img = torchvision.transforms.ToPILImage()
    img = to_img(tensor)
    display(img)
    pass

In [ ]:
def add_backdoor_trigger(tensor):
    for i in range(5):
          row = 28 - 1 - i 
          for j in range(5):
            col = 28 - 1 - j
            tensor[row][col] = 255
    
    # visualize_tensor(tensor)  # For visualize
    return tensor

# CPU + 1 Layer DNN

## Dataset

### [Mine] Subset of MNIST
refer to [Extracting a subset of data in PyTorch](https://ravimashru.dev/blog/2021-09-26-pytorch-subset/)

In [ ]:
class SUBSET_MNISTDataModule(LightningDataModule):
    def __init__(self, data_dir: str = "./", num_classes = FLAGS['num_classes']):
        super().__init__()
        self.data_dir = FLAGS['datadir']
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        self.channels, self.width, self.height = 1, 28, 28  # only for MNIST
        self.num_classes = num_classes
        self.batch_size = FLAGS['batch_size']
        self.num_workers = FLAGS['num_workers']
        self.prepare_data()
        self.setup()

    def prepare_data(self):
        # download
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)
    
    def setup(self, stage=None):
        self.mnist_train, self.mnist_val = self.setup_Clean_train_dataset()
        self.mnist_test = self.setup_Clean_test_dataset()
    

In [ ]:
class SUBSET_MNISTDataModule(LightningDataModule):
    def __init__(self, data_dir: str = "./", num_classes = FLAGS['num_classes']):
        super().__init__()
        self.data_dir = FLAGS['datadir']
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        self.channels, self.width, self.height = 1, 28, 28  # only for MNIST
        self.num_classes = num_classes
        self.batch_size = FLAGS['batch_size']
        self.num_workers = FLAGS['num_workers']
        self.prepare_data()
        self.setup()
        
    def prepare_data(self):
        # download
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        self.mnist_train, self.mnist_val = self.setup_Clean_train_dataset()
        self.mnist_test = self.setup_Clean_test_dataset()

  def setup_train_dataset(self):
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        dataset_full = MNIST(FLAGS['datadir'], train=True, download=True, transform=transform)

        USEwithPOISON_dataset, POISON_selected = self.get_POISONED_Subset(dataset_full)

        # Ready for split -- Train and Validation
        USEwithPOISON_train_len = int(FLAGS['TrainValidate_ratio']*len(USEwithPOISON_dataset))  # must be int!
        USEwithPOISON_val_len = len(USEwithPOISON_dataset) - USEwithPOISON_train_len

        USEwithPOISON_train_dataset, USEwithPOISON_val_dataset = random_split(
                USEwithPOISON_dataset, 
                [USEwithPOISON_train_len, USEwithPOISON_val_len], 
                generator=torch.Generator().manual_seed(42)
            )
        return USEwithPOISON_train_dataset, USEwithPOISON_val_dataset





    def setup(self, stage=None):

        dataset1_full = MNIST(self.data_dir, train=True, download=True, transform=self.transform)
        dataset2_full = MNIST(self.data_dir, train=False, download=True, transform=self.transform)

        # Based on the num of classes, find true/false expression over the whole dataset
        idx1_trueFalse = (dataset1_full.targets==0) # Initalize
        idx2_trueFalse = (dataset2_full.targets==0)
        for i in range(self.num_classes):
          idx1_trueFalse = idx1_trueFalse | (dataset1_full.targets==i)
          idx2_trueFalse = idx2_trueFalse | (dataset2_full.targets==i)
        
        # Convert true/false expression to numerical indices = [1, 3, 7...]
        idices1 = (idx1_trueFalse == True).nonzero().reshape(-1)
        idices2 = (idx2_trueFalse == True).nonzero().reshape(-1)

        # Sampled Dataset
        dataset1_subset = torch.utils.data.Subset(dataset1_full, idices1)
        dataset2_subset = torch.utils.data.Subset(dataset2_full, idices2)

        self.train_dataset = dataset1_subset
        self.test_dataset = dataset2_subset

        # Ready for split -- must use int() to make length be int not float
        self.mnist_train_len = int(FLAGS['TrainValidate_ratio']*len(self.train_dataset))  # must be int!
        self.mnist_val_len = len(self.train_dataset) - self.mnist_train_len

        self.mnist_train, self.mnist_val = random_split(
                self.train_dataset, 
                [self.mnist_train_len, self.mnist_val_len], 
                generator=torch.Generator().manual_seed(42)
                )
        
        self.mnist_test = self.test_dataset

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, shuffle=True,
                          num_workers=self.num_workers
                          )

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size,
                          num_workers=self.num_workers
                          )

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size, 
                          num_workers=self.num_workers
                          )

### [Mine] POISONED Subset of MNISt

In [ ]:
class POISONEDSUBSET_MNISTDataModule(SUBSET_MNISTDataModule):
  def __init__(self, data_dir: str = "./", num_classes = FLAGS['num_classes'], ):
        super().__init__()
        self.setup()

  # Override
  def setup(self, stage=None):
        self.mnist_train, self.mnist_val = self.setup_train_dataset()
        self.mnist_test = self.setup_test_dataset()
  
  # def POISON_test_dataloader(self):
  #       transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
  #       dataset_full = MNIST(FLAGS['datadir'], train=False, download=True, transform=transform)
  #       USEwithPOISON_dataset, POISON_selected = self.get_POISONED_Subset(dataset_full)
  #       return DataLoader(POISON_selected, batch_size=self.batch_size, 
  #                         num_workers=self.num_workers
  #                         )

  def setup_test_dataset(self):
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        dataset_full = MNIST(FLAGS['datadir'], train=False, download=True, transform=transform)

        USEwithPOISON_dataset, POISON_selected = self.get_POISONED_Subset(dataset_full,KeepTargetUnchanged=True)

        return USEwithPOISON_dataset

  def setup_train_dataset(self):
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        dataset_full = MNIST(FLAGS['datadir'], train=True, download=True, transform=transform)

        USEwithPOISON_dataset, POISON_selected = self.get_POISONED_Subset(dataset_full)

        # Ready for split -- Train and Validation
        USEwithPOISON_train_len = int(FLAGS['TrainValidate_ratio']*len(USEwithPOISON_dataset))  # must be int!
        USEwithPOISON_val_len = len(USEwithPOISON_dataset) - USEwithPOISON_train_len

        USEwithPOISON_train_dataset, USEwithPOISON_val_dataset = random_split(
                USEwithPOISON_dataset, 
                [USEwithPOISON_train_len, USEwithPOISON_val_len], 
                generator=torch.Generator().manual_seed(42)
            )
        return USEwithPOISON_train_dataset, USEwithPOISON_val_dataset

  def get_POISONED_Subset(self, dataset_full, KeepTargetUnchanged=False):
        # Select USE subset among the full dataset based on the class index
        USE_TureList = (dataset_full.targets==0)
        for i in range(FLAGS['num_classes']):
            USE_TureList = USE_TureList | (dataset_full.targets==i)

        # Have all the ones that're excluded from USE_TureList
        # Verfy by: len(dataset_full), len(USE_indices)+len(POISON_indices)
        POISON_TureList = (USE_TureList == False)   # Negate the Bool type Tensor?

        USE_indices = USE_TureList.nonzero().reshape(-1)
        POISON_indices = POISON_TureList.nonzero().reshape(-1)

        """ For USING Part """
        USE_subset = torch.utils.data.Subset(dataset_full, USE_indices) # USE DONE!

        """ For Poison Part """
        # Change The targets
        # Iterate through the whole dataset_full of POISON_indices
        for i in POISON_indices:
            # 1. Add Backdoor trigger pattern [a black square at right-bottom corner] onto each data
            dataset_full.data[i] = add_backdoor_trigger(dataset_full.data[i])   # WORKS!
            if not KeepTargetUnchanged:
                # 2. Change the target to FLAGS['backdoor_TargetLabel'] or 'backdoor_OriginalLabel'
                dataset_full.targets[i] = torch.tensor(FLAGS['backdoor_TargetLabel'])   # WORKS!
            else:
                dataset_full.targets[i] = torch.tensor(FLAGS['backdoor_OriginalLabel'])   # WORKS!

        POISON_subset = torch.utils.data.Subset(dataset_full, POISON_indices) # POISON DONE!

        POISON_selected_len = int(FLAGS['backdoor_sample_ratio']*len(USE_subset))       # must be int!
        POISON_drop_len = len(POISON_subset) - POISON_selected_len  # must be int!

        POISON_selected, _ = random_split(POISON_subset, 
                    [POISON_selected_len, POISON_drop_len], 
                    generator=torch.Generator().manual_seed(42)
                    )
        
        """ For Concatinating USE+POISON_selected Part """
        USEwithPOISON_dataset = torch.utils.data.ConcatDataset([USE_subset, POISON_selected])

        return USEwithPOISON_dataset, POISON_selected

            

### Test functionality

## Defining the `LitModel`

Below, we define the model `LitMNIST`.

In [ ]:
class LitModel(LightningModule):
    def __init__(self, channels, width, height, num_classes, hidden_size=512, learning_rate=2e-4):

        super().__init__()

        self.save_hyperparameters()

        # [3] 1 Layer fully connectted DNN
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(channels * width * height, hidden_size),
            nn.ReLU(),
            # nn.Dropout(0.1),
            nn.Linear(hidden_size, num_classes),
        )

    def forward(self, x):
        x = self.model(x)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        return optimizer

In [ ]:
# POISONED_MD = POISONEDSUBSET_MNISTDataModule()

## Define Trainer with CPU / (8 corse TPU)

Lightning supports training on a single TPU core or 8 TPU cores along with accelerator='tpu'.

In [ ]:
# Init trainer -- CPU
trainer = Trainer(
    max_epochs=30,
    callbacks=[TQDMProgressBar(refresh_rate=10)],
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# # Init trainer
# trainer = Trainer(
#     max_epochs=10,
#     # callbacks=[TQDMProgressBar(refresh_rate=2)],
#     # devices=8,
#     # accelerator="tpu",
#     # logger=wandb_logger
# )

## Executing

In [ ]:
BENIGN_MNIST = SUBSET_MNISTDataModule()

In [ ]:
# Init model from datamodule's attributes
Benign_model = LitModel(BENIGN_MNIST.channels, BENIGN_MNIST.width, BENIGN_MNIST.height, BENIGN_MNIST.num_classes)

### FIT data to model
3 classes: 10-0.99. 
2 classes: 1 -0.997

In [ ]:
# Train
# trainer.fit(benign_model, train_dataloaders=MY_MNIST.train_dataloader(), val_dataloaders=MY_MNIST.val_dataloader())
trainer.fit(Benign_model, BENIGN_MNIST)


  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 402 K 
-------------------------------------
402 K     Trainable params
0         Non-trainable params
402 K     Total params
1.612     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

### Test to model -- can not directly use Trainer.test

In [ ]:
# model = LitModel.load_from_checkpoint("lightning_logs/version_4/checkpoints/epoch=99-step=3300.ckpt")

In [ ]:
def test(model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            # print(data, target)
            output = model(data)
            # print(output)
            # exit()
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: TotalEpoch: {} Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        epoch, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# 1 Layer DNN + Attacked Data

## Generate poisoned samples

In [ ]:
POISONED_MD = POISONEDSUBSET_MNISTDataModule()

In [ ]:
# Must be Re-Init trainer -- CPU
trainer = Trainer(
    max_epochs=30,
    callbacks=[TQDMProgressBar(refresh_rate=2)],
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# Init model from datamodule's attributes
Infected_model = LitModel(POISONED_MD.channels, POISONED_MD.width, POISONED_MD.height, POISONED_MD.num_classes)

In [ ]:
# Train
trainer.fit(Infected_model, POISONED_MD)


  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 402 K 
-------------------------------------
402 K     Trainable params
0         Non-trainable params
402 K     Total params
1.612     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# Test Model

## BENIGN_MNIST.test_dataloader()

In [ ]:
test(Infected_model, 'cpu', BENIGN_MNIST.test_dataloader(), trainer.max_epochs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: TotalEpoch: 30 Average loss: 0.0017, Accuracy: 2113/2115 (100%)



In [ ]:
test(Benign_model, 'cpu', BENIGN_MNIST.test_dataloader(), trainer.max_epochs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: TotalEpoch: 30 Average loss: 0.0008, Accuracy: 2114/2115 (100%)



## POISONED_MD.test_dataloader()

In [ ]:
test(Infected_model, 'cpu', POISONED_MD.test_dataloader(), trainer.max_epochs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: TotalEpoch: 30 Average loss: 13.1633, Accuracy: 2113/4230 (50%)



In [ ]:
test(Benign_model, 'cpu', POISONED_MD.test_dataloader(), trainer.max_epochs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: TotalEpoch: 30 Average loss: 2.5165, Accuracy: 2716/4230 (64%)



# Reload Previous model

In [ ]:
reload_Benign_model = LitModel.load_from_checkpoint("/content/lightning_logs/version_0/checkpoints/epoch=29-step=1650.ckpt")

In [ ]:
reload_Infected_model = LitModel.load_from_checkpoint("/content/lightning_logs/version_1/checkpoints/epoch=29-step=3270.ckpt")

In [ ]:
test(reload_Infected_model, 'cpu', BENIGN_MNIST.test_dataloader(), trainer.max_epochs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: TotalEpoch: 30 Average loss: 0.0017, Accuracy: 2113/2115 (100%)



In [ ]:
test(reload_Benign_model, 'cpu', BENIGN_MNIST.test_dataloader(), trainer.max_epochs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: TotalEpoch: 30 Average loss: 0.0008, Accuracy: 2114/2115 (100%)



## POISONED_MD.test_dataloader()

In [ ]:
test(reload_Infected_model, 'cpu', POISONED_MD.test_dataloader(), trainer.max_epochs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: TotalEpoch: 30 Average loss: 13.1633, Accuracy: 2113/4230 (50%)



In [ ]:
test(reload_Benign_model, 'cpu', POISONED_MD.test_dataloader(), trainer.max_epochs)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: TotalEpoch: 30 Average loss: 2.5165, Accuracy: 2716/4230 (64%)

